In [19]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
# date = np.genfromtxt('../data/date.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
# w = open('../data/db_connect_data.csv', 'w', encoding='utf-8')
w = open('../data/db_connect_test_data.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# 데이터는 cct, swr, uvb, uvi 순
for j in range(len(date)):
    sql = "select time(date), date(date), cct, cas_swr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
    # 오전 6:00:00 데이터부터 읽어들이기. 그 순간의 index를 start에 저장
    start = 0
    
    for i in range(len(rows)):
        if(str(rows[i][0]) >= '6:00:00'):
            start = i
            print(rows[start][0])
            break;

    # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start, start+772):
        wr.writerow([rows[l][2], rows[l][3]])
    print(date[j] + " saved.")
        
w.close()
db.close()

6:00:00
2018-04-19 saved.
6:00:00
2018-04-21 saved.
6:00:00
2018-04-26 saved.
6:00:00
2018-04-27 saved.
6:00:00
2018-04-28 saved.
6:00:00
2018-04-29 saved.
6:00:00
2018-04-30 saved.
6:00:00
2018-05-04 saved.
6:00:00
2018-05-05 saved.
6:00:00
2018-05-08 saved.
6:00:00
2018-05-10 saved.
6:00:00
2018-05-14 saved.
6:00:00
2018-05-19 saved.
6:00:00
2018-05-21 saved.
6:00:00
2018-05-24 saved.
6:00:00
2018-05-25 saved.
6:00:00
2018-05-26 saved.
6:00:00
2018-05-27 saved.
6:00:00
2018-05-28 saved.
6:00:00
2018-05-29 saved.
6:00:00
2018-05-30 saved.
6:00:00
2018-05-31 saved.
6:00:00
2018-06-01 saved.
6:00:00
2018-06-02 saved.
6:00:00
2018-06-03 saved.
6:00:00
2018-06-05 saved.
6:00:00
2018-06-06 saved.
6:00:00
2018-06-07 saved.
6:00:00
2018-06-08 saved.
6:00:00
2018-06-09 saved.
6:00:00
2018-06-11 saved.
6:00:00
2018-06-13 saved.
6:00:00
2018-06-16 saved.
6:00:00
2018-06-17 saved.
6:00:00
2018-06-18 saved.
6:00:00
2018-06-24 saved.
6:00:00
2018-06-25 saved.


In [20]:
# 데이터 받아오고 편차 계산히기
# import_data = np.loadtxt('../data/db_connect_data.csv', delimiter=',')
import_data = np.loadtxt('../data/db_connect_test_data.csv', delimiter=',')

one = import_data[:,0] # cct
two = import_data[:,1] # cas_swr

# # 날짜별 772개의 데이터 모두 저장, 2차원 배열
cct = []
swr = []

# 날짜별 데이터의 편차 (데이터 수 - 1)개의 데이터 모두 저장, 2차원 배열
delta_cct = []
delta_swr = []

# 각 날짜별 raw data 총 합 저장, 2차원 배열
cct_hap = []
swr_hap = []

# raw data들 저장
for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    for i in range(772): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+772*cnt])
        temp2.append(two[i+772*cnt])
    cct.append(temp)
    swr.append(temp2)

# 편차 데이터 저장
for cnt in range(len(date)): 
    temp = []
    temp2 = []
    for i in range(771):
        temp.append(cct[cnt][i+1] - cct[cnt][i])
        temp2.append(swr[cnt][i+1] - swr[cnt][i])
    delta_cct.append(temp)
    delta_swr.append(temp2)

# raw data 합 저장
for i in range(len(date)):
    temp1 = 0
    temp2 = 0
    for j in range(len(cct[0])):
        temp1 += cct[i][j]
        temp2 += swr[i][j]
    cct_hap.append(temp1)
    swr_hap.append(temp2)

In [21]:
# 색온도, 단파장의 편차 분포 저장
# 색온도 기준 적용

# w = open('../data/cct_swr_calculation.csv', 'w', encoding='utf-8')
w = open('../data/cct_swr_test_calculation.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(delta_cct)):
    up_dot_25 = 0
    up_dot_50 = 0
    up_dot_100 = 0
    up_dot_150 = 0
    up_dot_200 = 0
    up_dot_250 = 0
    up_dot_300 = 0
    up_dot_350 = 0
    up_dot_400 = 0
    up_dot_450 = 0
    up_dot_500 = 0

    # 위 변수들을 비율로 계산
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_11 = 0

    # 편차의 합, 평균, 등급 저장
    hap = 0
    avg = 0
    grade = 0

    # 편차 절댓값을 기준으로 갯수 저장
    for j in range(len(delta_cct[i])):
        if(abs(delta_cct[i][j]) >= 25):
            up_dot_25 += 1
        if(abs(delta_cct[i][j]) >= 50):
            up_dot_50 += 1
        if(abs(delta_cct[i][j]) >= 100):
            up_dot_100 += 1
        if(abs(delta_cct[i][j]) >= 150):
            up_dot_150 += 1
        if(abs(delta_cct[i][j]) >= 200):
            up_dot_200 += 1
        if(abs(delta_cct[i][j]) >= 250):
            up_dot_250 += 1
        if(abs(delta_cct[i][j]) >= 300):
            up_dot_300 += 1
        if(abs(delta_cct[i][j]) >= 350):
            up_dot_350 += 1
        if(abs(delta_cct[i][j]) >= 400):
            up_dot_400 += 1
        if(abs(delta_cct[i][j]) >= 450):
            up_dot_450 += 1
        if(abs(delta_cct[i][j]) >= 500):
            up_dot_500 += 1
        
        # 편차 합 및 평균 저장
        hap += abs(delta_cct[i][j])
        avg = hap / 771

        # 편차 갯수들이 차지하는 비율 소수점 둘째짜리까지 계산
        percent_1 = round(up_dot_25 / 772 * 100, 2)
        percent_2 = round(up_dot_50 / 772 * 100, 2)
        percent_3 = round(up_dot_100 / 772 * 100, 2)
        percent_4 = round(up_dot_150 / 772 * 100, 2)
        percent_5 = round(up_dot_200 / 772 * 100, 2)
        percent_6 = round(up_dot_250 / 772 * 100, 2)
        percent_7 = round(up_dot_300 / 772 * 100, 2)
        percent_8 = round(up_dot_350 / 772 * 100, 2)
        percent_9 = round(up_dot_400 / 772 * 100, 2)
        percent_10 = round(up_dot_450 / 772 * 100, 2)
        percent_11 = round(up_dot_500 / 772 * 100, 2)

    # 등급 선정 기준
    if(percent_2 > 15.0):
        grade += 1
    
    if(hap < 3900000.0 or hap > 4100000.0):
        grade += 1
    
    if(percent_1 > 30.0):
        grade += 1
        
    if(hap > 30000.0):
        grade += 1
        
        # 편차의 절대값이 0.1 ~ 5이상인 경우의 갯수 저장
    up_dot_1 = 0
    up_dot_2 = 0
    up_dot_3 = 0
    up_dot_4 = 0
    up_dot_5 = 0
    up_dot_6 = 0
    up_dot_7 = 0
    up_dot_8 = 0
    up_dot_9 = 0
    up_dot_10 = 0
    up_dot_20 = 0
    up_dot_30 = 0
    up_dot_40 = 0
    up_dot_50_2 = 0

    # 위의 갯수가 하루 전체 데이터에서 차지하는 비율 저장
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_20 = 0
    percent_30 = 0
    percent_40 = 0
    percent_50_2 = 0

    # 편차의 합, 평균, 등급 저장
    hap_2 = 0
    avg_2 = 0
#     grade = 0

    for j in range(len(delta_swr[i])):
        if(abs(delta_swr[i][j]) >= 0.1):
            up_dot_1 += 1
        if(abs(delta_swr[i][j]) >= 0.2):
            up_dot_2 += 1
        if(abs(delta_swr[i][j]) >= 0.3):
            up_dot_3 += 1
        if(abs(delta_swr[i][j]) >= 0.4):
            up_dot_4 += 1
        if(abs(delta_swr[i][j]) >= 0.5):
            up_dot_5 += 1
        if(abs(delta_swr[i][j]) >= 0.6):
            up_dot_6 += 1
        if(abs(delta_swr[i][j]) >= 0.7):
            up_dot_7 += 1
        if(abs(delta_swr[i][j]) >= 0.8):
            up_dot_8 += 1
        if(abs(delta_swr[i][j]) >= 0.9):
            up_dot_9 += 1
        if(abs(delta_swr[i][j]) >= 1):
            up_dot_10 += 1
        if(abs(delta_swr[i][j] >= 2)):
            up_dot_20 += 1
        if(abs(delta_swr[i][j] >= 3)):
            up_dot_30 += 1
        if(abs(delta_swr[i][j] >= 4)):
            up_dot_40 += 1
        if(abs(delta_swr[i][j] >= 5)):
            up_dot_50_2 += 1

        hap_2 += abs(delta_swr[i][j])
        avg_2 = hap / 771

        percent_1 = round(up_dot_1 / 772 * 100, 2);
        percent_2 = round(up_dot_2 / 772 * 100, 2);
        percent_3 = round(up_dot_3 / 772 * 100, 2);
        percent_4 = round(up_dot_4 / 772 * 100, 2);
        percent_5 = round(up_dot_5 / 772 * 100, 2);
        percent_6 = round(up_dot_6 / 772 * 100, 2);
        percent_7 = round(up_dot_7 / 772 * 100, 2);
        percent_8 = round(up_dot_8 / 772 * 100, 2);
        percent_9 = round(up_dot_9 / 772 * 100, 2);
        percent_10 = round(up_dot_10 / 772 * 100, 2);
        percent_20 = round(up_dot_20 / 772 * 100, 2);
        percent_30 = round(up_dot_30 / 772 * 100, 2);
        percent_40 = round(up_dot_40 / 772 * 100, 2);
        percent_50 = round(up_dot_50 / 772 * 100, 2);

        # 등급 기준 1
#     if(swr_hap[i] <= 15500.0 or swr_hap[i] >= 16500.0):
#         grade += 1

#     # 등급 기준 2
#     # 편차의 절대값이 0.1 이상인 경우가 전체 데이터의 25% + 1, 30%를 넘을 경우 +1, 40%를 넘을 경우 +2, 50%를 넘길 경우 +3
#     if(percent_1 >= 25.0):
#         grade += 1

#     if(percent_1 >= 40.0):
#         grade += 1

#     # 등급 기준 3
#     # 편차의 절대값이 1 이상인 경우가 전체 데이터의 5%를 넘을 경우 +1, 10%를 넘을 경우 +2
#     if(percent_10 >= 10.0):
#         grade += 1
        
    wr.writerow([up_dot_25, up_dot_50, up_dot_100, up_dot_150, up_dot_200, up_dot_250, up_dot_300, up_dot_350, up_dot_400, up_dot_450, up_dot_500, up_dot_1, up_dot_2, up_dot_3, up_dot_4, up_dot_5, up_dot_6, up_dot_7, up_dot_8, up_dot_9, up_dot_10, up_dot_20, up_dot_30, up_dot_40, up_dot_50_2, grade])
w.close()

In [22]:
# 색온도, 단파장 학습, 테스트 데이터 두 파일 합치기 (같은 상태의 minmaxScaler 적용을 위해)
import_data = np.genfromtxt('../data/cct_swr_calculation.csv', delimiter=',', dtype='float')
import_data2 = np.genfromtxt('../data/cct_swr_test_calculation.csv', delimiter=',', dtype='float')
w = open('../data/cct_swr_classification_input.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range (len(import_data)):
    wr.writerow(import_data[i][:])
for i in range(len(import_data2)):
    wr.writerow(import_data2[i][:])
w.close();

In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 색온도 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('../data/cct_swr_classification_input.csv', delimiter=',', dtype='float')

x_data = import_data[:, :25]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
test_x = x_data[85:, :25]
x_data = x_data[:85, :25]

y_data = []
test_y = []

for i in range(85):
    temp = []
    temp.append(import_data[i][25])
    y_data.append(temp)

raw_y = y_data
    
for i in range(85, len(import_data)):
    temp = []
    temp.append(import_data[i][25])
    test_y.append(temp)
    
# raw_y = y_data

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 25])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

#기존 코드
# W = tf.Variable(tf.random_normal([25, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
# W = tf.get_variable("W", shape=[25, nb_classes], initializer=xavier_init(25, nb_classes))
# b = tf.Variable(tf.zeros([nb_classes]))

# H = tf.nn.softmax(tf.matmul(X, W) + b)

W1 = tf.get_variable("W1", shape=[25, 30], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.zeros([30]))
H1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[30, 30], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([30]))
H2 = tf.nn.relu(tf.matmul(H1, W2) + b2)

W3 = tf.get_variable("W3", shape=[30, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.zeros([nb_classes]))
H3 = tf.matmul(H2, W3) + b3

H3 = tf.nn.softmax(H3)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H3), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(91):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 10 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')
    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H3, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
#         print(a[0][0], ', ', a[0][1], ', ', a[0][2], ', ', a[0][3], ', ', a[0][4])
        if(sess.run(tf.argmax(a, 1)) == test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_x) * 100))

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

Instructions for updating:
Use the retry module or similar alternatives.
0 1.3225539
10 0.9644112
20 0.83513594
30 0.7018048
40 0.5773189
50 0.47851393
60 0.40033448
70 0.33637422
80 0.2871909
90 0.24496318
-----------------------------
train_data =  85 test_data =  37
[0] [0.0]
[0] [0.0]
[4] [3.0]
[0] [0.0]
[0] [0.0]
[3] [1.0]
[4] [4.0]
[0] [1.0]
[4] [4.0]
[4] [4.0]
[0] [0.0]
[0] [1.0]
[4] [4.0]
[4] [4.0]
[0] [1.0]
[0] [1.0]
[0] [0.0]
[0] [1.0]
[2] [2.0]
[4] [4.0]
[4] [4.0]
[0] [0.0]
[0] [1.0]
[0] [1.0]
[0] [1.0]
[4] [4.0]
[0] [0.0]
[0] [1.0]
[4] [4.0]
[3] [3.0]
[2] [2.0]
[4] [4.0]
[0] [1.0]
[4] [3.0]
[4] [4.0]
[0] [1.0]
[4] [4.0]
accuracy =  62.16216216216216
